<p style="text-align: center; font-size: 192%"> Computational Finance </p>
<img src="../img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 90%;">
<p style="text-align: center; font-size: 150%"> Week 6: Monte Carlo Methods </p>
<p style="text-align: center; font-size: 75%"> <a href="#copyrightslide">Copyright</a> </p>

In [1]:
#silence some warnings
import warnings
warnings.filterwarnings('ignore')

# Outline

* Solutions to exercises week 5
* Exam Information
    * Exam material
    * Example question
* Advanced Monte Carlo Methods

## Exam Information

*The following is also available on [Canvas](https://canvas.uva.nl/courses/25150/assignments/248824)*

* Time and date: **Thursday 20 December, 09:00-11:00**
* Location: **IWO 4.04A (Rood/Red)**
    * It is assumed that the exam will be on campus. If anything changes, this will be communicated via Canvas.
    * This is a written exam. While IWO allows for computer exams, you will *not* have access to a computer during the exam.
* The exam will be 2h, closed book, open questions, hand-written.
* Critical value tables will be available.
* Allowed:
    * Calculator
    * Cheat sheet: one page A4, double sided, hand-written

### Exam Material

* The relevant exam material is defined by the **lecture slides**. 

* The **reading list** in the slides for Week 1 is intended as background material to help you study, but won't be tested *per se*.

* **Tutorial notes** are background material as well -- unless they discuss material from the lecture slides.

* **Derivations** are intended as background knowledge. You should understand the intuition and the implication but you will not be asked to replicate derivations. However, you may be asked to apply methods such as Ito's lemma.

* The focus of the exam is on the financial (and statistical) theory from Weeks 3-6. You will not be required to write large functions. You may, however, be required to *read* **code** and explain what it does, with the following *exceptions*:

  * Week 1, *Advanced Material on Functions*.
  * Week 2, anything related to `pandas`, `panda_datareader`, and `statsmodels`; in other words, only the material on `numpy` is relevant.
  * All of Week 3 and 4. Note that only the programming part is excluded, not the financial theory. This means that while I don't expect you to be able to, e.g., reproduce the code for the regression of a factor model, I *do* expect you to be able to write down the regression equation, and interpret the regression output.
  
  The code discussed in Weeks 5-6, the exercises, and the second assigment are just applications of the material in Week 1 (plus `numpy`), so all these are fair game.
  


### Example question

*Question 5 from mock exam 1*

1. State the definition of a continuous-time martingale, and explain what it means.
2. Let 
    $$dX_t=\nu dt +\sigma dW_t, \quad \nu=-\frac{1}{2}\sigma^2,$$ 
    a Brownian motion with drift. For each of the following functions, derive the SDE satisfied by $Y_t\equiv f(t, X_t)$, and state whether $Y_t$ is a martingale.
    1. $f(t, X_t)=\exp(X_t)$;
    2. $f(t, X_t)=tX_t$.

#### Answers

1. $X_t$ is a martingale if
    * $\mathbb{E}[|X_t|]<\infty$
    * $\mathbb{E}[X_t\mid \mathcal F_{s}]=X_{s}$, $s<t$.

  This means that a martingale is a process without a drift; on average, we expect tomorrow's value to be equal to today's.
  
2. 
    1. $f(t, X_t)=\exp(X_t)$, so $\dot f(t, X_t)\equiv \tfrac{\partial}{\partial t}f(t, X_t)=0$, $f'(t, X_t)\equiv \tfrac{\partial}{\partial X_t}f(t, X_t)=\exp(X_t)$, and $f''(t, X_t)\equiv \tfrac{\partial^2}{\partial X_t^2}f(t, X_t)=\exp(X_t)$, so by Ito's lemma,
\begin{align*}
dY_t\equiv df(t,X_{t})&=\dot{f}(t,X_{t})dt+f^{\prime }(t,X_{t})dX_{t}+\frac{1}{2}f^{\prime \prime }(t,X_{t})\sigma^2dt\\
&=0+\exp(X_t)dX_{t}+\frac{1}{2}\exp(X_t)\sigma^2dt\\
&=\exp(X_t)(\nu dt +\sigma dW_t)+\frac{1}{2}\exp(X_t)\sigma^2dt\\
&=\exp(X_t)(-\frac{1}{2}\sigma^2 dt +\sigma dW_t)+\frac{1}{2}\exp(X_t)\sigma^2dt\\
&= \exp(X_t)\sigma dW_t\\
&= Y_t\sigma dW_t,
\end{align*}
an Ito process without a drift and hence a martingale.

    2. $f(t, X_t)=tX_t$, so $\dot f(t, X_t)=X_t$, $f'(t, X_t)=t$, $f''(t, X_t)=0$,  and
\begin{align*}
dY_t\equiv df(t,X_{t})&=\dot{f}(t,X_{t})dt+f^{\prime }(t,X_{t})dX_{t}+\frac{1}{2}f^{\prime \prime }(t,X_{t})\sigma^2dt\\
&=X_tdt+tdX_t+0\\
&=X_tdt+t(\nu dt +\sigma dW_t)\\
&=(X_t+\nu t)dt+\sigma t dW_t,\\
&=(X_t-\frac{1}{2}\sigma^2 t)dt+\sigma t dW_t,
\end{align*}
which has a non-zero drift and thus isn't a martingale.

## Advanced Monte Carlo Methods

*Preview of bonus week 6 slides on Canvas*
    
### Variance Reduction Techniques
* In standard Monte Carlo, the length of the confidence interval for $\theta\equiv\mathbb{E}[X]$ is proportional to $\hat{\sigma}/\sqrt{n}$, where $\sigma$ is the standard deviation of $X$.
* Thus to increase the accuracy by a factor of 10 (i.e., gain 1 digit), we need 100 times as many samples.
* Variance reduction techniques aim to improve the accuracy of the estimate, without increasing $n$.
* Techniques to do so include: *antithetic sampling*, and *control variates*

In [10]:
from scipy.stats import norm
def asianmc_vec(S0, K, T, r, sigma, q, N, numsim=10000):
    """Monte Carlo price of an arithmetic average Asian call."""
    X0 = np.log(S0)
    nu = r-q-.5*sigma**2    
    #simulate all paths at once:
    _, X = bmsim_vec(T, N, X0, nu, sigma, numsim)
    S = np.exp(X)
    payoffs = np.maximum(S[:, 1:].mean(axis=1)-K, 0.)  #S[1:]->S[:, 1:], max->maximum, mean()->mean(axis=1)
    g = np.exp(-r*T)*payoffs
    C = g.mean(); s = g.std()
    zq = norm.ppf(0.975)
    Cl = C - zq/np.sqrt(numsim)*s
    Cu = C + zq/np.sqrt(numsim)*s
    return C, Cl, Cu

### Greeks in Monte Carlo

* So far we were mostly concerned with *pricing* options. An equally important problem in practice is *hedging*:
suppose that a financial institution has issued a call option (i.e., it is short the option). In order to eliminate the risk, it might create the hedge
$$
\Pi _{t}\equiv-C_{t}+\phi _{t}S_{t},
$$
where $\phi_{t}$ is a number of stocks. The *delta-hedged* portfolio has
$$
\phi _{t}=\Delta_t\equiv \frac{\partial C_{t}}{\partial S_{t}}.
$$
* This position is *delta-neutral*: it is immune to (infinitesimally) small movements of the underlying.
* The portfolio has to be rebalanced every time the underlying moves (because $\Delta_t$ depends on time), incurring trading costs.
* $\Delta_t$ is an example of a so-called option *Greek*. Other examples are $\Gamma_t$ (second derivative w.r.t. the underlying), $\theta_t$ (the derivative with respect to time), $\rho_t$ (w.r.t. $r$), and $\mathcal{V}_t$ ("Vega", w.r.t. $\sigma$).

* If the option cannot be priced analytically, then we may need to resort to Monte Carlo.
* Recalling our risk neutral pricing formula, we see that
$$
\Delta_0=\frac{\partial}{\partial S_0} \mathbb{E}^{\mathbb{Q}}\left[e^{-rT}C_T(S_0)\right].
$$
* In view of the definition of the derivative,
$$
\Delta_0\equiv \lim_{\Delta S_0\rightarrow 0} \frac{\mathbb{E}^{\mathbb{Q}}\left[e^{-rT}C_T(S_0+\Delta S_0)\right]-\mathbb{E}^{\mathbb{Q}}\left[e^{-rT}C_T(S_0)\right]}{\Delta S_0},
$$
it is natural to approximate $\Delta_0$ with a finite difference quotient for some small $\delta S_0$:

In [12]:
S0 = 11; K = 10; T = 3/12.; r = 0.02; sigma = .3; q = 0.01; N = 10; numsim=10000
dS = .01   # change from 0.01 to 0.001 to see differences
np.random.seed(0)
Cd, _, _ = asianmc_vec(S0+dS, K, T, r, sigma, q, N, numsim)
C,  _, _ = asianmc_vec(S0,    K, T, r, sigma, q, N, numsim)
Delta = (Cd-C)/dS; Delta   # True answer is approx. 0.858

7.886858496600313

* Ouch. What's happening is that as $\Delta S_0\rightarrow 0$, the sampling variation in $C_T$ increasingly dominates the variation from a small perturbation of $S_0$. This increases the variance of the estimator (which is nevertheless unbiased as $\Delta S_0\rightarrow 0$).
* The trick is to use the same random numbers in both simulation runs:

In [13]:
dS = .001; np.random.seed(0)
Cd, _, _ = asianmc_vec(S0+dS, K, T, r, sigma, q, N, numsim)
np.random.seed(0)
C,  _, _ = asianmc_vec(S0,    K, T, r, sigma, q, N, numsim)
Delta = (Cd-C)/dS; Delta

0.8622451045858792

* This is called the *method of common random numbers*.
* It implies changing the order of the limit and the expectation.
* This can be made more explicit using a *pathwise estimate*, where you directly sample the derivative.
* The pathwise estimate allows construction of a confidence interval for $\Delta_0$.

<section id="copyrightslide">

# Copyright Statement
* Course slides were created by Simon Broda for Python 2.7 $-$ Andreas Rapp adapted them to Python 3.6. Maintained and updated by Bart Keijsers.
* This work is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/).
* All figures have been produced for this course using Python. Empirical results are based on public data available from [FRED](https://fred.stlouisfed.org/), [Quandl/WIKI](https://www.quandl.com/databases/WIKIP), Kenneth French's website, and [Yahoo Finance](https://finance.yahoo.com/).
* More information on Simon Broda's [Github](https://github.com/s-broda/ComputationalFinance/blob/master/LICENSE.md).